In [434]:
import pandas as pd
import os
import openpyxl
import numpy as np
import re
from datetime import datetime
import requests
import shutil


In [584]:
class scrapeSECData:
    def __init__(self,ticker):
        self.ticker = ticker 
        self.CIK = self.getCIK()
        self.filings, self.fiscalYear = self.getSECFilings()
        self.writeFilings()
        
        
    def getCIK(self): #get SEC code for company based on ticker
        headers={"User-Agent": "Mozilla/5.0"}
        symbol_to_cik = requests.get("https://www.sec.gov/files/company_tickers.json").json() #returns a json dictionary with a indexed list of all different companies 
        ciks = {info["ticker"]:info["cik_str"] for key,info in symbol_to_cik.items()} #create dictionary indexable by ticker
        return ciks[self.ticker]
    
        
    
    def getSECFilings(self):
        headers={"User-Agent": "Mozilla/5.0"}
        edgar_filings = requests.get(f"https://data.sec.gov/submissions/CIK{self.CIK:0>10}.json", headers=headers).json()
        filings = pd.DataFrame(edgar_filings["filings"]["recent"])
        filings = filings.loc[filings["reportDate"]>"2014-01-01"]#filings pre-2014 are unable to be downloaded due to different excel format
        
        #get 10Q and 10k filings
        filings = filings.loc[filings["form"].isin(["10-Q","10-K"])].reset_index(drop=True) #drop all filings that are not 10Q or 10K
        filings["Month"] = pd.to_datetime(filings["reportDate"]).dt.strftime("%m").astype("int") #get Month of all filings 
        filings["Year"] = pd.to_datetime(filings["reportDate"]).dt.strftime("%Y").astype("int") #get Year of all filings
        
#         filings = filings.sort_values(by="reportDate",ascending=False)
        
        #get a fiscal Year of data by finding the last 10K and the three 10Qs before it
        TenKIndex = filings.loc[filings["form"]=="10-K"].index[0] 
        fiscalYearKey = filings.loc[TenKIndex:TenKIndex+3,["Month","form"]].copy().reset_index(drop=True)
        fiscalYearKey["Period"] = ["Year Ended", "Q3","Q2","Q1"]
        
        
        #fiscal year data for certain period can vary between months ie. both Jan & Feb, therefore I found all possible months of filings and found appropriate period for each month
        fiscalYear = pd.DataFrame({"Month":filings.Month.unique()})
        fiscalYear.index = list(fiscalYear.Month.apply(lambda x: np.argmin(np.abs(fiscalYearKey.Month-x)))) #using fiscal year key, I found the index of the period in fiscalYearKey with closest month to each month 
        fiscalYear = pd.merge(fiscalYear,fiscalYearKey, left_index=True, right_index=True, suffixes=("","_x")) #I merged together together all of the possible months with their corresponding period based on the key
        fiscalYear = fiscalYear.drop(columns="Month_x")
        
        print(fiscalYear)
        #merge fiscal year labels with filings by the month that they were reported
        filings = pd.merge(filings,fiscalYear,on=["Month","form"],how="left")
        
        if filings["Period"].isnull().any():
            print("WARNING THERE IS A FILING WITH A NULL FISCAL PERIOD LABEL")
        
        #set Fiscal Year
        filings["Fiscal Year"] = filings["Year"]
        #get Quarters that are a part of different fiscal Year than report date
        TenKMo = fiscalYear.loc[fiscalYear["form"]=="10-K"]["Month"].iloc[0]
        filings.loc[(filings["Month"]>TenKMo)&(filings["form"]!="10-K"),"Fiscal Year"] += 1 #the 10K is reported in a month before the 10Q so therefore the 10Q is the next fiscal year 
        
        return filings, fiscalYear
        
        
    def writeFilings(self):
        #create a folder to put all of the filings in
        path = os.path.join("input",f"Financial Statement {self.ticker}")
        folder = os.path.join(path,"raw")
        if f"Financial Statement {self.ticker}" not in os.listdir("input"):
            os.mkdir(path)
        if "raw" in os.listdir(path):
            shutil.rmtree(folder)
        os.mkdir(folder)
            
        
        #iterate through each filing, grab the accession number to access the filing and write to excel file
        for i,row in self.filings.iterrows():
            period = row["Period"]
            fiscalYear = row["Fiscal Year"]
            Year = row["Year"]
            accessionNum = row["accessionNumber"].replace("-","")
            
            url = f"https://www.sec.gov/Archives/edgar/data/{self.CIK}/{accessionNum}/Financial_Report.xlsx"
            req = requests.get(url,headers={"User-Agent": "Mozilla/5.0"})
            
            #only include the parenthesis actual year if the fiscal year is different than the actual filing date
            if fiscalYear != Year:
                name = os.path.join(folder,f"{period} {fiscalYear} ({Year}).xlsx")
            else: 
                name = os.path.join(folder,f"{period} {fiscalYear}.xlsx")

            file = open(name,"wb")
            file.write(req.content)
            
        self.fiscalYear.to_csv(os.path.join(path,"fiscalYear.csv"))
    
        

In [585]:
scrapeSECData("GPS").filings

   Month  form      Period
0      1  10-K  Year Ended
0      2  10-K  Year Ended
1     10  10-Q          Q3
1     11  10-Q          Q3
2      7  10-Q          Q2
2      8  10-Q          Q2
3      4  10-Q          Q1
3      5  10-Q          Q1


,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription,Month,Year,Period,Fiscal Year
0,0000039911-22-000037,2022-05-27,2022-04-30,2022-05-27T13:08:51.000Z,34,10-Q,001-07562,22975424,,5563526,1,1,gps-20220430.htm,10-Q,4,2022,Q1,2023
1,0000039911-22-000012,2022-03-15,2022-01-29,2022-03-15T12:38:35.000Z,34,10-K,001-07562,22739987,,12328211,1,1,gps-20220129.htm,10-K,1,2022,Year Ended,2022
2,0000039911-21-000132,2021-11-24,2021-10-30,2021-11-24T13:05:19.000Z,34,10-Q,001-07562,211444291,,6626321,1,1,gps-20211030.htm,10-Q,10,2021,Q3,2022
3,0000039911-21-000108,2021-08-27,2021-07-31,2021-08-27T12:39:18.000Z,34,10-Q,001-07562,211216524,,8283388,1,1,gps-20210731.htm,10-Q,7,2021,Q2,2022
4,0000039911-21-000063,2021-05-28,2021-05-01,2021-05-28T12:41:11.000Z,34,10-Q,001-07562,21978541,,5858925,1,1,gps-20210501.htm,10-Q,5,2021,Q1,2022
5,0000039911-21-000021,2021-03-16,2021-01-30,2021-03-16T14:50:49.000Z,34,10-K,001-07562,21745242,,11984519,1,1,gps-20210130.htm,10-K,1,2021,Year Ended,2021
6,0000039911-20-000139,2020-11-25,2020-10-31,2020-11-25T13:15:02.000Z,34,10-Q,001-07562,201349086,,7244595,1,1,gps-20201031.htm,10-Q,10,2020,Q3,2021
7,0000039911-20-000108,2020-08-31,2020-08-01,2020-08-31T15:51:55.000Z,34,10-Q,001-07562,201151532,,6909025,1,1,q2202010-qdocument.htm,10-Q,8,2020,Q2,2021
8,0000039911-20-000069,2020-06-09,2020-05-02,2020-06-09T14:17:36.000Z,34,10-Q,001-07562,20951533,,5920540,1,1,q1202010-qdocument.htm,10-Q,5,2020,Q1,2021
9,0000039911-20-000019,2020-03-17,2020-02-01,2020-03-17T17:10:10.000Z,34,10-K,001-07562,20721912,,14496566,1,1,fy201910-k.htm,10-K,2,2020,Year Ended,2020


In [579]:
class FinancialReport():
    def __init__(self,filePath):
        self.filePath = filePath
        self.readFile()

    def readFile(self):
        self.statements = pd.read_excel(self.filePath,sheet_name=None)
    
    def parseSheet(self, sheet):
        sheet.columns = ["label"]+list(sheet.columns[1:])
        
        sheet = sheet.loc[~sheet.label.isnull(), sheet.columns.notna()]
        
        #remove rows where the label is just [1] or [2] because they are empty
        sheet = sheet.loc[sheet.label.str.contains("[a-zA-Z]",regex=True)]
       
        if not sheet.iloc[:,1].astype("string").str.isnumeric().any():
            sheet=sheet.drop(sheet.columns[1],axis=1) #delete columns where there is only one value ie. a note like [1]
        
        sheet = sheet.iloc[:,:2] 
        return sheet
        
    def findBalanceSheet(self):
        #find sheet where the first sheet value (header) includes balance
        balanceSheets = [sheet for name,sheet in self.statements.items() if "balance" in str(sheet.columns[0]).lower()]
        balance = balanceSheets[0]
        
        balance = self.parseSheet(balance)
        
        return balance
        
    def findIncomeSheet(self):
        #find sheet where the first sheet value (header) includes income
        incomeSheets = [sheet for name,sheet in self.statements.items() if "income" in str(sheet.columns[0]).lower() or "operations" in str(sheet.columns[0]).lower()]
        income = incomeSheets[0]
        
        #remove the first row which doesn't say the date but instead says the length of time period ie. 3 mo ended 
        income.columns = income.iloc[0]
        income = income.drop(0)
        
        income = self.parseSheet(income)
        return income

        
    def findCashflowSheet(self): 
        #find sheet where the first sheet value (header) includes income
        cashSheets = [sheet for name,sheet in self.statements.items() if "cash" in str(sheet.columns[0]).lower()]
        cashflow = cashSheets[0]
        
        #remove the first row which doesn't say the date but instead says the length of time period ie. 3 mo ended 
        cashflow.columns = cashflow.iloc[0]
        cashflow = cashflow.drop(0)
        
        cashflow = self.parseSheet(cashflow)
        return cashflow
        
        

        
   
    

In [580]:
x = FinancialReport("input/Financial Statement XOM/raw/Year Ended 2017.xlsx")
x.findBalanceSheet()

,label,"Dec. 31, 2017"
0,Current assets,NaN
1,Cash and cash equivalents,3177
2,"Notes and accounts receivable, less estimated ...",25597
3,Inventories,NaN
4,"Crude oil, products and merchandise",12871
5,Materials and supplies,4121
6,Other current assets,1368
7,Total current assets,47134
8,"Investments, advances and long-term receivables",39160
9,"Property, plant and equipment, at cost, less a...",252630


In [586]:
class compileFilings:
    def __init__(self,ticker,fromDate):
        self.ticker = ticker 
        self.fromDate = fromDate
        self.path = os.path.join("input",f"Financial Statement {self.ticker}")
        self.folder = os.path.join(self.path,"raw")
        
        self.filings = self.getFilings()
        self.filings = self.getFilingsPostDate(self.filings)
        self.files = self.getFilePathsFromFilings(self.filings)
        
        
        self.writeIncomeExcel()
        self.writeCashflowExcel()
        self.writeBalanceExcel()
    
    def getFilings(self): #get the names of all of the filings
        #get a list of all of the filings 
        filings = os.listdir(self.folder)
        if ".DS_Store" in filings:
            filings.remove(".DS_Store")
        filings = [filing for filing in filings if"~" not in filing]
        
        return filings
    
    def getYears(self,quarters): #go through each file name and find the year of filing
        return [int(re.findall("\d{4}",quarter)[0]) for quarter in quarters]
    
    
    def getPossibleFilingsFromYears(self, years):
        return [q + " " + str(year) for year in years for q in ["Q1","Q2","Q3","Q4","Year Ended"]]
    
    def getFilingsPostDate(self,filings):
        years = range(self.fromDate,2023)
        names = self.getPossibleFilingsFromYears(years)
        names = ["Year Ended "+str(self.fromDate-1)]+names
        files = [filing for name in names for filing in filings if name in filing] 
        return files
    
    def getFilePathsFromFilings(self,filings): #take file names and return a list of the paths to the file
        return [os.path.join(self.folder,filing) for filing in filings]
        
    def writeIncomeExcel(self):
        
        file = os.path.join(self.path, f"Income Statements All-{self.ticker}.xlsx")
        Income = pd.ExcelWriter(file)
        
        #for each filing, read the filing into a dataframe and save to a combined excel file
        for file in self.files:
            print(file)
            statement = FinancialReport(file)
            income = statement.findIncomeSheet()
            name = file.split("/")[-1].split(".")[0]
            income.to_excel(Income, index=False, sheet_name=name)
        
        Income.save()
        
    def writeCashflowExcel(self):
        
        file = os.path.join(self.path, f"Cashflow Statements All-{self.ticker}.xlsx")
        Cashflow = pd.ExcelWriter(file)
        #for each filing, read the filing into a dataframe and save to a combined excel file
        for file in self.files:
            statement = FinancialReport(file)
            cashflow = statement.findCashflowSheet()
            name = file.split("/")[-1].split(".")[0]
            cashflow.to_excel(Cashflow, index=False, sheet_name=name)
        
        Cashflow.save()
        
    
    def writeBalanceExcel(self):
        
        file = os.path.join(self.path, f"Balance Statements All-{self.ticker}.xlsx")
        Balance = pd.ExcelWriter(file)
        #for each filing, read the filing into a dataframe and save to a combined excel file
        for file in self.files:
            statement = FinancialReport(file)
            balance = statement.findBalanceSheet()
            name = file.split("/")[-1].split(".")[0]
            balance.to_excel(Balance, index=False, sheet_name=name)
        
        Balance.save()
        
        

In [587]:
scrapeSECData("GPS")
compileFilings("GPS",2017)

   Month  form      Period
0      1  10-K  Year Ended
0      2  10-K  Year Ended
1     10  10-Q          Q3
1     11  10-Q          Q3
2      7  10-Q          Q2
2      8  10-Q          Q2
3      4  10-Q          Q1
3      5  10-Q          Q1
input/Financial Statement GPS/raw/Year Ended 2016.xlsx
input/Financial Statement GPS/raw/Q1 2017 (2016).xlsx
input/Financial Statement GPS/raw/Q2 2017 (2016).xlsx
input/Financial Statement GPS/raw/Q3 2017 (2016).xlsx
input/Financial Statement GPS/raw/Year Ended 2017.xlsx
input/Financial Statement GPS/raw/Q1 2018 (2017).xlsx
input/Financial Statement GPS/raw/Q2 2018 (2017).xlsx
input/Financial Statement GPS/raw/Q3 2018 (2017).xlsx
input/Financial Statement GPS/raw/Year Ended 2018.xlsx
input/Financial Statement GPS/raw/Q1 2019 (2018).xlsx
input/Financial Statement GPS/raw/Q2 2019 (2018).xlsx
input/Financial Statement GPS/raw/Q3 2019 (2018).xlsx
input/Financial Statement GPS/raw/Year Ended 2019.xlsx
input/Financial Statement GPS/raw/Q1 2020 (2019).xl